In [1]:
!pip install deeplake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.6/512.6 kB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB

In [2]:
import deeplake

import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [3]:
ds_train = deeplake.load("hub://activeloop/animal10n-train")
ds_test = deeplake.load("hub://activeloop/animal10n-test")

x_data = ds_train['images']
x_data = np.reshape(x_data, (x_data.shape[0], 64, 64, 3))
x_data = x_data.astype('float32') / 255.0

y_data = to_categorical(ds_train["labels"])

x_test = ds_test['images']
x_test = np.reshape(x_test, (x_test.shape[0], 64, 64, 3))
x_test = x_test.astype('float32') / 255.0

y_test = to_categorical(ds_test["labels"])

\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/animal10n-train



-

hub://activeloop/animal10n-train loaded successfully.



|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/animal10n-test



\

hub://activeloop/animal10n-test loaded successfully.



In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_data, y_data, batch_size=256, epochs=15, validation_data=(x_test, y_test))

Epoch 1/15
196/196 [==============================] - 430s 2s/step - loss: 1.9878 - accuracy: 0.2800 - val_loss: 1.5929 - val_accuracy: 0.4500
Epoch 2/15
196/196 [==============================] - 419s 2s/step - loss: 1.6486 - accuracy: 0.4094 - val_loss: 1.4375 - val_accuracy: 0.5056
Epoch 3/15
196/196 [==============================] - 422s 2s/step - loss: 1.5556 - accuracy: 0.4453 - val_loss: 1.4037 - val_accuracy: 0.5048
Epoch 4/15
196/196 [==============================] - 449s 2s/step - loss: 1.4911 - accuracy: 0.4678 - val_loss: 1.3200 - val_accuracy: 0.5404
Epoch 5/15
196/196 [==============================] - 452s 2s/step - loss: 1.4417 - accuracy: 0.4832 - val_loss: 1.2546 - val_accuracy: 0.5586
Epoch 6/15
196/196 [==============================] - 450s 2s/step - loss: 1.3952 - accuracy: 0.4998 - val_loss: 1.2082 - val_accuracy: 0.5836
Epoch 7/15
196/196 [==============================] - 453s 2s/step - loss: 1.3694 - accuracy: 0.5084 - val_loss: 1.2469 - val_accuracy: 0.5588

In [5]:
loss, accuracy = model.evaluate(x_test, y_test)

print(f"Accuracy: {accuracy}")

157/157 [==============================] - 11s 68ms/step - loss: 1.0650 - accuracy: 0.6296
Accuracy: 0.6295999884605408


In [6]:
model.save('second_model_try_hw13.h5')